In [ ]:
#If you’re happy with the dataset, then load it
from datasets import load_dataset
from transformers import AutoTokenizer

from itertools import islice
from typing import Iterable, List, Tuple

import argparse
import numpy as np
import torch
import datasets
from datasets.dataset_dict import DatasetDict

from gensim.models.keyedvectors import KeyedVectors
from torch import nn
import random

https://medium.com/dataseries/k-fold-cross-validation-with-pytorch-and-sklearn-d094aa00105f
Cross validation

https://medium.com/analytics-vidhya/text-summarization-using-nlp-3e85ad0c6349
text summarization

https://medium.com/analytics-vidhya/text-summarization-using-bert-gpt2-xlnet-5ee80608e961
More text summarization


PREPROCESSING DECISIONS:

1. Labels at every tenth percentile.

2. Max input sequence length - both standard BERT and max-doc-length as separate models.

3. New-lines removed.

In [ ]:
# Load datasets
tifu_short_raw = load_dataset("reddit_tifu", "short", split="train")
tifu_long_raw = load_dataset("reddit_tifu", "long", split = "train")

In [ ]:
# Add decimal labels to upvote ratio
def add_columns(example):
    example["upvote_ratio"] = round(example["upvote_ratio"], 1)
    example["labels"] = int(example["upvote_ratio"] * 10)
    return example

tifu_short = tifu_short_raw.map(add_columns)

tifu_long = tifu_long_raw.map(add_columns)

In [ ]:
# Filter dataset_short for rows without a tldr

tifu_no_tldr = tifu_short.filter(lambda example: example["tldr"] == "")

len(tifu_no_tldr) == len(tifu_short) - len(tifu_long) # True. Yay!

In [ ]:

# Create smaller chunks of the no_tldr dataset

small_no_tldr_1 = tifu_no_tldr.select(list(range(10000)))
small_no_tldr_2 = tifu_no_tldr.select(list(range(10000, 20000)))
small_no_tldr_3 = tifu_no_tldr.select(list(range(20000, len(tifu_no_tldr))))

print(len(small_no_tldr_1), len(small_no_tldr_2), len(small_no_tldr_3))
# Generate tldr for 10.000 posts
from summarizer import TransformerSummarizer, Summarizer
bert_model = Summarizer()

# Add decimal labels to upvote ratio
def gen_tldr(example):
    raw_text = example["documents"]
    cleaner_text = raw_text.replace("\n\n", '.')
    clean_text = raw_text.replace('\n', '.')
    bert_summary = ''.join(bert_model(clean_text, num_sentences = 2))
    example["tldr"] = bert_summary
    return example


In [ ]:
# Generate the 1st 10.000 tldr's.
gen_tldr_1 = small_no_tldr_1.map(gen_tldr)
gen_tldr_1.save_to_disk("preprocessed/gen_tldr_1")

In [ ]:
#Generate the 2nd 10.000 tldr's.

gen_tldr_2 = small_no_tldr_2.map(gen_tldr)
gen_tldr_2.save_to_disk("preprocessed/gen_tldr_2")



In [ ]:
random.seed(1337)

# Split into train, validation and test.
dict1 = tifu_short.train_test_split(test_size=0.1)

train_and_val = dict1["train"]
test_short = dict1["test"]

dict2 = train_and_val.train_test_split(test_size=0.2)

train_short = dict2["train"]
val_short = dict2["test"]

In [ ]:
# Tokenize
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenization(example):
    return tokenizer(example["documents"], padding = True, truncation= True, return_tensors = "pt")


train_input = train_short.map(tokenization, batched = True)
val_input = val_short.map(tokenization, batched = True)
test_input = test_short.map(tokenization, batched = True)

In [ ]:
train_input.save_to_disk("preprocessed/train_short")
val_input.save_to_disk("preprocessed/val_short")
test_input.save_to_disk("preprocessed/test_short")

In [ ]:
# Let's train a model!

def prepare_batch(token_ids, labels):
    token_id_tensor = torch.LongTensor(np.array(token_ids)) # Turn list of lists into np.array into torch.LongTensor
    labels = torch.LongTensor(np.array(labels))
    output = tuple([token_id_tensor, labels])
    return output

model_name = "bert-base-uncased"

from datasets import load_from_disk

train_cl_short = load_from_disk("../preprocessed/train_short")
val_cl_short = load_from_disk("../preprocessed/val_short")
test_cl_short = load_from_disk("../preprocessed/test_short")

from torch.utils.data import DataLoader

train_cl_short = train_cl_short.remove_columns(['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title'])
train_cl_short.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

small_train = train_cl_short.select(range(1000))

train_dataloader = torch.utils.data.DataLoader(small_train, batch_size=32)

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=11)

from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)

nosave = 0

epochs = 2

In [ ]:
print("[INFO:] Training classifier...")

k = 0
for epoch in range(epochs):
    ## for each batch
    for batch in train_dataloader:
        k += 1
        print("This is working for the ", k, "th time!")
        ## train on one batch
            # FORWARD PASS
        y = model(**batch)
            # Calculate loss
        loss = y.loss
            # backpropagation
        loss.backward()
            # take step, reset
        optimizer.step()
        optimizer.zero_grad()
        # Validate model
    #val_X = val_idx
    #val_y = model(val_X)
        # Calculate validation loss
    #val_loss = model.loss_fn(outputs=val_y, labels=val_tags)
        # If best_val_loss exists, see if it needs updating.
    #try:
    #    if best_val_loss > val_loss.item():
    #        best_val_loss = val_loss.item()
    #            # save the model if it is the best so far
    #        torch.save(model, f"models/{model_name}")
    #        nosave = 0
    #    else:
    #            # Increment nosave
    #        nosave += 1
        # Otherwise, create it and save the model.
    #except NameError:
    #    best_val_loss = val_loss.item()
    #    torch.save(model, f"models/{model_name}")
    #    # stop the training if patience is used up
    #if nosave > 5:
    #    print("Patience is up!")
    #    break
        # some print to see that it is running
        #if (epoch + 1) % 10 == 0: # Comment or uncomment depending on how many epochs you want printed.
    print(f"epoch: {epoch+1}, loss = {loss.item():.4f}")
    # "best validation loss = {best_val_loss:.4f}")

print("[INFO:] Finished traning!")
